In [2]:
!pip install shap


   ---------------------------------------- 0.0/545.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/545.3 kB ? eta -:--:--
   ------------------- -------------------- 262.1/545.3 kB ? eta -:--:--
   ---------------------------------------- 545.3/545.3 kB 1.1 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import nltk
import joblib
import shap
import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\syeol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
import pandas as pd
df = pd.read_csv(r"F:\Fake review Project\fake reviews dataset.csv")
print(df.columns)



Index(['category', 'rating', 'label', 'text_'], dtype='object')


In [14]:
# Rename for clarity (optional)
df.rename(columns={'text_': 'reviewText'}, inplace=True)

# Convert label: CG = 1 (real), others = 0
df['label'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

In [15]:
# Select only what's needed
df = df[['reviewText', 'label']]
df.dropna(inplace=True)
df['reviewText'] = df['reviewText'].astype(str)

# Custom features
df['length'] = df['reviewText'].apply(len)
df['exclaim_count'] = df['reviewText'].apply(lambda x: x.count('!'))
df['upper_case_ratio'] = df['reviewText'].apply(
    lambda x: sum(1 for w in x.split() if w.isupper()) / len(x.split()) if len(x.split()) > 0 else 0
)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Vectorization (limit features to avoid memory crash)
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=5000)
X_text = tfidf.fit_transform(df['reviewText'])

# 2. Custom features as sparse matrix
X_custom_sparse = csr_matrix(df[['length', 'exclaim_count', 'upper_case_ratio']].values)

# 3. Combine text + custom
X_all = hstack([X_text, X_custom_sparse])
y = df['label']

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

# 5. Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
joblib.dump(model, 'model.pkl')



c:\Users\syeol\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\syeol\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

['model.pkl']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
joblib.dump(model, 'model.pkl')
